In [30]:
import pandas as pd
import xml.etree.ElementTree as et
from sklearn.preprocessing import OneHotEncoder
import numpy as np
from numpy import asarray
from numpy import savez_compressed

In [32]:
tree=et.parse("Distillation.xml")
root=tree.getroot()

In [54]:
cause={}
tag={}
n_tag=[]
n_cause=[]
cause_index_map={}

In [55]:
i=0
for x in root.findall("Causes/CauseDTO"):
    id_=x.find("Id")
    desc=x.find("Description")
    cause[id_.text]=desc.text
    n_cause.append(desc.text)
    cause_index_map[desc.text]=i
    i=i+1

In [57]:
savez_compressed('cause_index_map.npz',cause_index_map)

In [35]:
for x in root.findall('ProcessElements/ProcessElementDTO'):
    id_=x.find("Id")
    desc=x.find("Point")
    n_tag.append(desc.text)
    tag[id_.text]=desc.text

In [36]:
df=pd.DataFrame(columns=n_tag)
df.insert(0, 'Causes', n_cause)
df.set_index('Causes', inplace=True)

In [37]:
for rel in root.findall('Relations/CauseEffectRelationDTO'):
    ri=rel.find("Cause_Id").text
    ci=rel.find("ProcessElement_Id").text
    relation=rel.find("Relation").text
    df.at[cause[ri],tag[ci]]=int(relation)

In [42]:
df=df.fillna(0)
df.head(10)

,11LC16,11TI21,11LC14,11FC15,11TI24,11PI14,11TI26,11TI22,11FC19,11PC15,11FC17,11PC16,11TI20,11TI25,11FC20,11AC12
Causes,,,,,,,,,,,,,,,,
11AT12: Overhead C5 composition analyser fails high,-2,0,1,1,0,0,-1,0,2,0,1,1,0,-2,-2,2
11HS14: Overhead condenser fans fail off,-2,-1,1,1,-1,1,1,0,-1,2,2,2,0,-2,-2,-2
11PV15: Overhead pressure control valve fails closed,-2,0,2,2,-1,2,0,0,1,0,-2,0,0,0,0,0
11FV15: Rebolier condensate flow control valve fails closed,-2,-1,2,-2,-1,-1,0,-1,0,-1,-2,-1,-1,-1,-1,-2
11PV16A: Reflux drum pressure control valve fails closed,0,0,0,0,0,0,-1,0,0,0,0,-2,0,0,0,-1
11FT19: Reflux flow transmitter fails low,-2,0,2,2,-2,-1,-2,0,-2,0,1,1,-1,-2,-2,-2
P-62: Reflux transfer pump fails off,2,0,-1,-1,1,-1,1,0,-2,0,-1,-1,0,2,2,1
P-68: Top product transfer pump fails off,2,0,0,0,0,0,0,0,0,0,0,0,0,-2,-2,0
11FV17: Bottom product transfer to tank 1 valve fails closed,1,0,2,-1,0,0,0,0,1,0,-2,0,0,0,0,0


In [43]:
encoder = OneHotEncoder()
n_runs=[]
for column in df.columns:
    dic={}
    keys=df[column].unique().tolist()
    tag=column
    dic["tag"]=tag
    keys.sort()
    effects=[-2,-1,0,1,2]
    tag_matrix=pd.DataFrame(columns=effects)
    encoder_df = pd.DataFrame(encoder.fit_transform(df[[column]]).toarray(),columns=keys)
    for column in encoder_df.columns:
        tag_matrix[column]=encoder_df[column]
    # encoder_df
    tag_matrix.fillna(0)
    arr=tag_matrix.to_numpy()
    count = np.count_nonzero(arr == 1, axis=0).reshape((5,1)) ##along column
    effect=-2
    for i in count:
        if(i):
            dic[effect]=1
        else:
            dic[effect]=0
        effect=effect+1
        
    n_runs.append(dic)
    arr = np.divide(np.transpose(arr), count, out=np.zeros_like(np.transpose(arr)), where=count!=0)
    arr=np.transpose(arr)
    savez_compressed('p_matrices/'+str(tag)+'.npz', arr)
    

In [45]:
savez_compressed('params.npz',n_runs)

# Input required
1-Connect it with the application.

2-Every time it is run, record the following:

    a) Tag which is getting affected
    b) Effect (Nature of increase in tag)
    c) Actual Cause

# Working 
1- Base matrices(probablity of particular cause for a given effect) will be stored in some server

2- Everytime application is run, real time data has to be collected,and probability will be updated accordingly

In [60]:
# for checking data loaded in npz f
from numpy import load
dict_data = load('cause_index_map.npz',allow_pickle=True)
data = dict_data['arr_0']
print(data)

{'11AT12: Overhead C5 composition analyser fails high': 0, '11HS14: Overhead condenser fans fail off': 1, '11PV15: Overhead pressure control valve fails closed': 2, '11FV15: Rebolier condensate flow control valve fails closed': 3, '11PV16A: Reflux drum pressure control valve fails closed': 4, '11FT19: Reflux flow transmitter fails low': 5, 'P-62: Reflux transfer pump fails off': 6, 'P-68: Top product transfer pump fails off': 7, '11FV17: Bottom product transfer to tank 1 valve fails closed': 8}
